In [3]:
n = 3
factor = 3
K = [Symbol(f'K_{i+1}') for i in range(n)]
D = Function('D')
h = Symbol('h')

In [8]:
def getFactor(expr, n):
    i = 0
    while K[i] not in expr.free_symbols:
        i += 1
    if i == n:
        return None
    e = expr.diff(K[i])
#     display(e)
    ls = e.as_ordered_factors()
#     display(ls)
    numcheck = lambda k: k.is_number
    vals = np.array(list(filter(numcheck, ls)))
    if len(vals) == 0:
        return 1
    vals = functools.reduce(lambda a, b: a * b, vals)
#     print(vals)
    return sympify(1 / vals)
    
    
def extrapolate(a, b, n):
    A = getFactor(a, n)
    B = getFactor(b, n)
    display(a, b)
    factor = B / A
    display(factor * b, factor*b - a)
    return (factor * b - a) / (factor - 1)

In [10]:
def richardsons(main_term, err_coefs, n, step, factor):
    terms =  functools.reduce(lambda a, b: a + b, [err_coefs[i] * h ** (i+1) for i in range(len(err_coefs))])
    data = np.zeros((n, n), dtype='object')
    data[0][0] = main_term(step) + terms
    for i in range(1, n):
        step /= factor
        data[0][i] = main_term(step) + terms.subs(h, step)
    
    display(data)
    
    for i in range(1, n):
        for j in range(i, n):
            display((i, j))
            data[i][j] = extrapolate(data[i-1][j-1], data[i-1][j], n)
            display(data[i][j])
    
    return data
data = richardsons(lambda a: (1 + a) ** (1/a),  K, n, h, 2)

array([[K_1*h + K_2*h**2 + K_3*h**3 + (h + 1)**(1/h),
        K_1*h/2 + K_2*h**2/4 + K_3*h**3/8 + (h/2 + 1)**(2/h),
        K_1*h/4 + K_2*h**2/16 + K_3*h**3/64 + (h/4 + 1)**(4/h)],
       [0, 0, 0],
       [0, 0, 0]], dtype=object)

(1, 1)

K_1*h + K_2*h**2 + K_3*h**3 + (h + 1)**(1/h)

K_1*h/2 + K_2*h**2/4 + K_3*h**3/8 + (h/2 + 1)**(2/h)

K_1*h + K_2*h**2/2 + K_3*h**3/4 + 2*(h/2 + 1)**(2/h)

...

-K_2*h**2/2 - 3*K_3*h**3/4 + 2*(h/2 + 1)**(2/h) - (h + 1)**(1/h)

-K_2*h**2/8 - 3*K_3*h**3/32 + 2*(h/4 + 1)**(4/h) - (h/2 + 1)**(2/h)

-K_2*h**2/2 - 3*K_3*h**3/8 + 8*(h/4 + 1)**(4/h) - 4*(h/2 + 1)**(2/h)

3*K_3*h**3/8 + 8*(h/4 + 1)**(4/h) - 6*(h/2 + 1)**(2/h) + (h + 1)**(1/h)

K_3*h**3/8 + 8*(h/4 + 1)**(4/h)/3 - 2*(h/2 + 1)**(2/h) + (h + 1)**(1/h)/3

In [11]:
data

array([[K_1*h + K_2*h**2 + K_3*h**3 + (h + 1)**(1/h),
        K_1*h/2 + K_2*h**2/4 + K_3*h**3/8 + (h/2 + 1)**(2/h),
        K_1*h/4 + K_2*h**2/16 + K_3*h**3/64 + (h/4 + 1)**(4/h)],
       [0,
        -K_2*h**2/2 - 3*K_3*h**3/4 + 2*(h/2 + 1)**(2/h) - (h + 1)**(1/h),
        -K_2*h**2/8 - 3*K_3*h**3/32 + 2*(h/4 + 1)**(4/h) - (h/2 + 1)**(2/h)],
       [0, 0,
        K_3*h**3/8 + 8*(h/4 + 1)**(4/h)/3 - 2*(h/2 + 1)**(2/h) + (h + 1)**(1/h)/3]],
      dtype=object)

In [6]:
math.e

2.718281828459045

In [17]:
lstr = ''
for i in range(n):
    s, k = h, 0.04
    lstr += f' N_{i+1}({s}) &= {latex(eq_round(data[i][i]))}' + r'\\'
    lstr += """\n"""
    lstr += f' N_{i+1}({k}) &= {latex(eq_round(data[i][i].subs(h, k)))}' + r'\\'
    lstr += """\n"""
    s /= 2
    k /= 2
print(lstr)

 N_1(h) &= K_{1} h + K_{2} h^{2} + K_{3} h^{3} + \left(h + 1\right)^{\frac{1}{h}}\\
 N_1(0.04) &= 0.04 K_{1} + 0.0016 K_{2} + 6.4 \cdot 10^{-5} K_{3} + 2.665836\\
 N_2(h) &= - \frac{K_{2} h^{2}}{2} - \frac{3 K_{3} h^{3}}{4} + 2 \left(\frac{h}{2} + 1\right)^{\frac{2}{h}} - \left(h + 1\right)^{\frac{1}{h}}\\
 N_2(0.04) &= - 0.0008 K_{2} - 4.8 \cdot 10^{-5} K_{3} + 2.71734\\
 N_3(h) &= \frac{K_{3} h^{3}}{8} + \frac{8 \left(\frac{h}{4} + 1\right)^{\frac{4}{h}}}{3} - 2 \left(\frac{h}{2} + 1\right)^{\frac{2}{h}} + \frac{\left(h + 1\right)^{\frac{1}{h}}}{3}\\
 N_3(0.04) &= 8.0 \cdot 10^{-6} K_{3} + 2.718273\\



In [14]:
for i in range(len(data)):
    for j in range(i, len(data)):
        display(data[i][j].subs(h, 0.04))

0.04*K_1 + 0.0016*K_2 + 6.4e-5*K_3 + 2.66583633148742

0.02*K_1 + 0.0004*K_2 + 8.0e-6*K_3 + 2.69158802907361

0.01*K_1 + 0.0001*K_2 + 1.0e-6*K_3 + 2.70481382942153

-0.0008*K_2 - 4.8e-5*K_3 + 2.71733972665979

-0.0002*K_2 - 6.0e-6*K_3 + 2.71803962976945

8.0e-6*K_3 + 2.718272930806